<a href="https://colab.research.google.com/github/HannaK79/ab_test_analyzer/blob/main/ab_test_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Створення універсального інструменту для аналізу A/B-тестів у Python
Мета проєкту — створити універсальний Python-скрипт для автоматичного аналізу результатів A/B-тестів: обчислення конверсій, статистичної значущості змін і формування підсумкової таблиці результатів для подальшої візуалізації.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from google.colab import drive



drive.flush_and_unmount()
drive.mount("/content/drive")


ab_test_data = pd.read_csv("/content/drive/MyDrive/python_tasks/bq-results.csv")


ab_test_data.head()


Mounted at /content/drive


,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-02,Ecuador,mobile,Americas,Organic Search,2,2,session with orders,1
1,2020-11-02,Kuwait,desktop,Asia,Organic Search,2,2,session with orders,1
2,2020-11-06,Cyprus,desktop,Asia,Direct,2,1,session with orders,1
3,2020-11-06,Ecuador,desktop,Americas,Paid Search,2,1,session with orders,1
4,2020-11-07,Tunisia,desktop,Africa,Direct,2,1,session with orders,1


In [ ]:
# для розрахунку конверсій
def count_metric(total, partial):
    return partial / total * 100

metrics = ["add_payment_info", "add_shipping_info", "begin_checkout", "new account"]

results = []

# Групування по тестах
for test_num in ab_test_data["test"].unique():

    # загальна кількість сесій у групі A і B
    total_a = ab_test_data[
        (ab_test_data["test_group"] == 1) &
        (ab_test_data["event_name"] == "session") &
        (ab_test_data["test"] == test_num)
    ]["value"].sum()

    total_b = ab_test_data[
        (ab_test_data["test_group"] == 2) &
        (ab_test_data["event_name"] == "session") &
        (ab_test_data["test"] == test_num)
    ]["value"].sum()

    # перебираємо метрики
    for metric in metrics:

        # кількість подій (event_name = metric) у групах A і B
        partial_a = ab_test_data[
            (ab_test_data["test_group"] == 1) &
            (ab_test_data["event_name"] == metric) &
            (ab_test_data["test"] == test_num)
        ]["value"].sum()

        partial_b = ab_test_data[
            (ab_test_data["test_group"] == 2) &
            (ab_test_data["event_name"] == metric) &
            (ab_test_data["test"] == test_num)
        ]["value"].sum()

        # конверсії (%)
        conv_A = count_metric(total_a, partial_a)
        conv_B = count_metric(total_b, partial_b)

        # пропорції для статистики
        p_A = partial_a / total_a
        p_B = partial_b / total_b
        p_pool = (partial_a + partial_b) / (total_a + total_b)

        # Z-статистика та p-value
        z_stat = (p_A - p_B) / np.sqrt(p_pool * (1 - p_pool) * (1/total_a + 1/total_b))
        p_value = 2 * (1 - stats.norm.cdf(abs(z_stat)))

        # зберігаємо у список
        results.append({
            "test_number": test_num,
            "metric": metric,
            "events_A": partial_a,
            "sessions_A": total_a,
            "conversion_A": conv_A,
            "events_B": partial_b,
            "sessions_B": total_b,
            "conversion_B": conv_B,
            "metric_change": (conv_B - conv_A) / conv_A * 100,
            "z_stat": z_stat,
            "p_value": p_value,
            "significant": p_value < 0.05
        })

# результати в таблицю
df_results = pd.DataFrame(results).sort_values(by="test_number")
df_results

,test_number,metric,events_A,sessions_A,conversion_A,events_B,sessions_B,conversion_B,metric_change,z_stat,p_value,significant
7,1,new account,3823,45362,8.427759,3681,45193,8.145067,-3.354299,1.542883,0.122859,False
6,1,begin_checkout,3784,45362,8.341784,4021,45193,8.897396,6.660587,-2.978783,0.002894,True
5,1,add_shipping_info,3034,45362,6.688418,3221,45193,7.127210,6.560481,-2.603571,0.009226,True
4,1,add_payment_info,1988,45362,4.382523,2229,45193,4.932180,12.542021,-3.924884,0.000087,True
3,2,new account,4165,50637,8.225211,4184,50244,8.327362,1.241934,-0.588793,0.556000,False
2,2,begin_checkout,4262,50637,8.416770,4313,50244,8.584110,1.988164,-0.952898,0.340642,False
1,2,add_shipping_info,3480,50637,6.872445,3510,50244,6.985909,1.650995,-0.709557,0.477979,False
0,2,add_payment_info,2344,50637,4.629026,2409,50244,4.794602,3.576911,-1.240994,0.214608,False
15,3,new account,5856,70047,8.360101,5822,70439,8.265308,-1.133880,0.643489,0.519907,False
14,3,begin_checkout,9532,70047,13.608006,9264,70439,13.151805,-3.352445,2.511389,0.012026,True


In [ ]:
# вигрузка файлу в csv
df_results.to_csv("/content/drive/MyDrive/python_tasks/ab_test_results.csv", index=False)

In [ ]:
# зберігання на комп'ютері
from google.colab import files
files.download("/content/drive/MyDrive/python_tasks/ab_test_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://public.tableau.com/app/profile/hanna.kontar/viz/A_Btest_17606143495870/ABtest?publish=yes
 посилання на візуалізовані і пораховані конверсії у табло